# Drone Routing Optimization Problem

## Model Initialization

In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
import numpy as np
#import pandas as pd
#import time
#from datetime import date

# initialize model object
mymodel = glp.Solver('Drone Routing Problem', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

### Import distance matrix

In [2]:
# import csv distance matrix as list
with open('Orders_Data.csv', newline='') as f:
    reader = csv.reader(f)
    distance = list(reader)

# remove blanks
for i in range(len(distance)):
    distance[i] = [item for item in distance[i] if item]
distance = [item for item in distance if item]
distance = np.array(distance, dtype = 'float')

# Cost per unit of distance
truck_cost_per_dist = 10.0
drone_cost_per_dist = 3.0

# Create cost matrices for truck & drone
truck_cost = distance * truck_cost_per_dist
drone_cost = distance * drone_cost_per_dist
truck_cost = truck_cost.tolist()
drone_cost = drone_cost.tolist()

In [3]:
len(distance)

6

In [4]:
truck_cost

[[0.0, 1000.0, 810.0, 990.0, 610.0, 1110.0],
 [1000.0, 0.0, 970.0, 980.0, 410.0, 930.0],
 [810.0, 970.0, 0.0, 190.0, 660.0, 360.0],
 [990.0, 980.0, 190.0, 0.0, 740.0, 180.0],
 [610.0, 410.0, 660.0, 740.0, 0.0, 770.0],
 [1110.0, 930.0, 360.0, 180.0, 770.0, 0.0]]

In [5]:
drone_cost

[[0.0, 300.0, 243.0, 297.0, 183.0, 333.0],
 [300.0, 0.0, 291.0, 294.0, 123.0, 279.0],
 [243.0, 291.0, 0.0, 57.0, 198.0, 108.0],
 [297.0, 294.0, 57.0, 0.0, 222.0, 54.0],
 [183.0, 123.0, 198.0, 222.0, 0.0, 231.0],
 [333.0, 279.0, 108.0, 54.0, 231.0, 0.0]]

### Initialize Model Parameters

In [6]:
# model parameters
N = len(distance)
M = N + 100 # Large number
D = 2 # Drones onboard the truck (0 to 2 index)

In [7]:
list(range(N))

[0, 1, 2, 3, 4, 5]

In [8]:
np.zeros((6,6))

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

## Decision Variables

In [9]:
# Truck path from node i to j
truck_arc = np.zeros((N,N)).tolist() # create nested list
for i in range(N):
    for j in range(N):
        truck_arc[i][j] = mymodel.IntVar(0, 1, "truck_" + str(i) + "." + str(j)) #create binary variable for each possible truck path
truck_arc

[[truck_0.0, truck_0.1, truck_0.2, truck_0.3, truck_0.4, truck_0.5],
 [truck_1.0, truck_1.1, truck_1.2, truck_1.3, truck_1.4, truck_1.5],
 [truck_2.0, truck_2.1, truck_2.2, truck_2.3, truck_2.4, truck_2.5],
 [truck_3.0, truck_3.1, truck_3.2, truck_3.3, truck_3.4, truck_3.5],
 [truck_4.0, truck_4.1, truck_4.2, truck_4.3, truck_4.4, truck_4.5],
 [truck_5.0, truck_5.1, truck_5.2, truck_5.3, truck_5.4, truck_5.5]]

In [10]:
mymodel.NumVariables()

36

In [11]:
# The kth drone path from node i to j

drone_arc = np.zeros((N,N)).tolist() # create nested list

for i in range(N):
    for j in range(N):
        drone_arc[i][j] = mymodel.IntVar(0, 1, 'drone_' + str(i) + "." + str(j)) #create binary variable for each possible drone path

        #for k in D: [k]  + "." + str(k) , int(len(D))
        
# All drone paths from node 2 (index 1)
drone_arc

[[drone_0.0, drone_0.1, drone_0.2, drone_0.3, drone_0.4, drone_0.5],
 [drone_1.0, drone_1.1, drone_1.2, drone_1.3, drone_1.4, drone_1.5],
 [drone_2.0, drone_2.1, drone_2.2, drone_2.3, drone_2.4, drone_2.5],
 [drone_3.0, drone_3.1, drone_3.2, drone_3.3, drone_3.4, drone_3.5],
 [drone_4.0, drone_4.1, drone_4.2, drone_4.3, drone_4.4, drone_4.5],
 [drone_5.0, drone_5.1, drone_5.2, drone_5.3, drone_5.4, drone_5.5]]

In [12]:
mymodel.NumVariables()

72

In [13]:
# create position variable (lambda / u)
pos = list(range(N))
for i in range(N):
    pos[i] = mymodel.IntVar(1, N-1, "p." + str(i))
pos[0].SetLb(0)
pos[0].SetUb(0)

In [14]:
mymodel.NumVariables()

78

## Objective

In [15]:
# create objective function
optimal_route = mymodel.Objective()
optimal_route.SetMinimization() # Minimize cost

for i in range(N):
    for j in range(N):
        if i != j: optimal_route.SetCoefficient(truck_arc[i][j], truck_cost[i][j]) # Truck cost per arc
        
for i in range(N):
    for j in range(N):
        if i != j: optimal_route.SetCoefficient(drone_arc[i][j], drone_cost[i][j]) # Drone cost per arc


## Constraints

In [16]:
first_arc = mymodel.Constraint(1,1)
for j in range(1,N):
    first_arc.SetCoefficient(truck_arc[0][j], 1)

In [17]:
last_arc = mymodel.Constraint(1,1)
for i in range(1,N):
    last_arc.SetCoefficient(truck_arc[i][0], 1)

In [18]:
mymodel.NumConstraints()

2

In [19]:
bal = list(range(N))
for j in range(N):
    bal[j] = mymodel.Constraint(0, 0)
    for i in range(N):
        bal[j].SetCoefficient(truck_arc[i][j], 1)
        bal[j].SetCoefficient(truck_arc[j][i], -1)

In [20]:
mymodel.NumConstraints()

8

In [21]:
launch = np.zeros(N).tolist()
for i in range(1,N):
    launch[i] = mymodel.Constraint(-D, 0)
    for j in range(1,N):
        launch[i].SetCoefficient(drone_arc[i][j], 1)
    for k in range(N):
        launch[i].SetCoefficient(truck_arc[k][i], -D)

In [22]:
mymodel.NumConstraints()

13

In [23]:
coverage = list(range(N))
for j in range(1,N):
    coverage[j] = mymodel.Constraint(1, 1)
    for i in range(N):
        if i!=j: coverage[j].SetCoefficient(truck_arc[i][j], 1)
    for i in range(1,N):
        coverage[j].SetCoefficient(drone_arc[i][j], 1)

In [24]:
mymodel.NumConstraints()

18

In [25]:
MTZ = np.zeros((N,N)).tolist() # create nested list

for i in range(N):
    for j in range(1,N):
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[i][j].SetCoefficient(truck_arc[i][j], N-1)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [26]:
mymodel.NumConstraints()

48

In [27]:
# Solve the model and print optimal solution
#start_time = time.time()
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % optimal_route.Value())

#print("Solution Time: %.2f seconds" % (time.time() - start_time))
#runtime = round(time.time() - start_time)
#today = str(date.today())

# Display optimal solution
for i in range(N):
    print('pos[%d] = %d' % (i, pos[i].solution_value()))
    for j in range(N):
        if truck_arc[i][j].solution_value() == 1:
            print('Truck ', i, ' to ', j, ': ', truck_arc[i][j].solution_value(), sep = '')
for i in range(1,N):
    for j in range(1,N):
        if drone_arc[i][j].solution_value() == 1:
                print('Drone from ', i, ' to ', j, ': ', drone_arc[i][j].solution_value(), sep = '')
        #for k in D:

Solution Status = 0
Number of variables = 78
Number of constraints = 48
Optimal Solution:
Optimal Value = 2368.00
pos[0] = 0
Truck 0 to 4: 1.0
pos[1] = 1
pos[2] = 2
Truck 2 to 0: 1.0
pos[3] = 1
pos[4] = 1
Truck 4 to 2: 1.0
pos[5] = 2
Drone from 2 to 3: 1.0
Drone from 2 to 5: 1.0
Drone from 4 to 1: 1.0
